In [1]:
#Use the NBA API to gather player stats from this season as well as last season
%pip install nba_api
%pip install pandas



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import teams
import pandas as pd

In [3]:
print(pd.__version__)

2.2.3


In [4]:
career = playercareerstats.PlayerCareerStats(player_id="203999")

career.get_data_frames()[0]

career.get_json()

career.get_dict()

{'resource': 'playercareerstats',
 'parameters': {'PerMode': 'Totals', 'PlayerID': 203999, 'LeagueID': None},
 'resultSets': [{'name': 'SeasonTotalsRegularSeason',
   'headers': ['PLAYER_ID',
    'SEASON_ID',
    'LEAGUE_ID',
    'TEAM_ID',
    'TEAM_ABBREVIATION',
    'PLAYER_AGE',
    'GP',
    'GS',
    'MIN',
    'FGM',
    'FGA',
    'FG_PCT',
    'FG3M',
    'FG3A',
    'FG3_PCT',
    'FTM',
    'FTA',
    'FT_PCT',
    'OREB',
    'DREB',
    'REB',
    'AST',
    'STL',
    'BLK',
    'TOV',
    'PF',
    'PTS'],
   'rowSet': [[203999,
     '2015-16',
     '00',
     1610612743,
     'DEN',
     21.0,
     80,
     55,
     1733.0,
     307,
     600,
     0.512,
     28,
     84,
     0.333,
     154,
     190,
     0.811,
     181,
     379,
     560,
     189,
     79,
     50,
     104,
     208,
     796],
    [203999,
     '2016-17',
     '00',
     1610612743,
     'DEN',
     22.0,
     73,
     59,
     2038.0,
     494,
     854,
     0.578,
     45,
     139,
     0.

In [ ]:
nba_teams = teams.get_teams()
#Select dictionary for the Nuggets, which contains their team ID
lakers = [team for team in nba_teams if team['abbreviation'] == 'LAL'][0]
nuggets_id = nuggets['id']

In [6]:
from nba_api.stats.endpoints import leaguegamefinder

#Query for games the Nuggets were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = nuggets_id)
#we  want the first dataframe that is returned
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612743,DEN,Denver Nuggets,0022400822,2025-02-24,DEN @ IND,W,240,125,...,0.786,14,32,46,39,8,4,19,21,9.0
1,22024,1610612743,DEN,Denver Nuggets,0022400808,2025-02-22,DEN vs. LAL,L,238,100,...,0.762,12,33,45,28,6,1,20,18,-23.0
2,22024,1610612743,DEN,Denver Nuggets,0022400794,2025-02-20,DEN vs. CHA,W,241,129,...,0.750,16,39,55,29,3,3,13,14,14.0
3,22024,1610612743,DEN,Denver Nuggets,0022400780,2025-02-12,DEN vs. POR,W,240,132,...,0.792,12,37,49,26,9,7,12,20,11.0
4,22024,1610612743,DEN,Denver Nuggets,0022400763,2025-02-10,DEN vs. POR,W,239,146,...,0.900,6,32,38,44,12,2,8,23,29.0


In [7]:
#get the last 10 games that the Nuggets played in 2025 
nuggets_game = games[(games['SEASON_ID'] == '22024') & (games['TEAM_ID'] == nuggets_id)].head(10)






In [8]:
warriors = [team for team in nba_teams if team['abbreviation'] == 'GSW'][0]
warriors_id = warriors['id']
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = warriors_id)
games = gamefinder.get_data_frames()[0]
warriors_games = games[(games['SEASON_ID'] == '22024') & (games['TEAM_ID'] == warriors_id)]




In [9]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable


In [10]:
def format_team_stats(team_df):
    "Format team stats for OpenAi"
    wins = team_df['WL'].value_counts()['W']
    losses = team_df['WL'].value_counts()['L']
    average_points = team_df['PTS'].mean()
    win_percentage = (wins/ (wins + losses)) * 100

    stats_summary = f"""
    2025 statistics:
    -Wins and Losses: {wins} - {losses}
    -Average Points scored: {average_points:.1f}
    -Win Percentage: {win_percentage:.1f}%

    Game by Game Results:
    """

    for _, game in team_df.iterrows():
        stats_summary += f"\n{game['GAME_DATE']}: {game['MATCHUP']} - {game['WL']} ({game['PTS']})"

    return stats_summary, team_df['TEAM_NAME']


In [11]:
import os 
import openai

In [12]:
openai.api_key  = os.getenv('open_ai_key')

In [ ]:
def get_prediction(team1,team2):
    """Generate a prediction from OpenAI for a game between two teams"""

    prompt = f"""Based on the following NBA statistics, predict the winner and final score of a game between {team1[1][0]} and {team2[1][0]}:
    
    {team1[1][0]} Stats:
    {team1[0]}

    {team2[1][0]} Stats:
    {team2[0]}

    Please provide your prediction in this following format:
    Winner: [team1 name]
    Score: [team2 name][score] - [team2 name][score]
    Confidence: [percentage]
    Brief explanation:[1-2 sentences]

    """

    try:
        response = openai.chat.completions.create(
            model = "gpt-4o-mini",
            messages = [{"role": "system",
                         "content": "You are a professional NBA analyst with expertise in predicting game outcomes based on team statistics"},
                        {"role": "user",
                         "content": prompt}],
                         temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error getting prediction: {str(e)}"


In [ ]:
nuggets_stats = format_team_stats(nuggets_game)
lakers_stats = format_team_stats(lakers_games)
prediction = get_prediction(nuggets_stats, warriors_stats)

prediction

"Winner: Denver Nuggets  \nScore: Nuggets 128 - Warriors 110  \nConfidence: 85%  \nBrief explanation: The Denver Nuggets are on a hot streak with a remarkable win percentage of 90.0% and an impressive average of 124.2 points scored. In contrast, the Golden State Warriors, while competitive, have struggled with consistency, averaging only 110.6 points per game, which likely won't be enough to keep pace with the high-scoring Nuggets."